<a href="https://colab.research.google.com/github/keesterbrugge/rl_intro_sutton/blob/main/ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example 4.2: Jack’s Car Rental Jack manages two locations for a nationwide car
rental company. Each day, some number of customers arrive at each location to rent cars.
If Jack has a car available, he rents it out and is credited 10 by the national company.
If he is out of cars at that location, then the business is lost. Cars become available for
renting the day after they are returned. To help ensure that cars are available where
they are needed, Jack can move them between the two locations overnight, at a cost of
$2 per car moved. We assume that the number of cars requested and returned at each
location are Poisson random variables, meaning that the probability that the number is n is  n e  , where   is the expected number. Suppose   is 3 and 4 for rental requests at n!
the first and second locations and 3 and 2 for returns. To simplify the problem slightly, we assume that there can be no more than 20 cars at each location (any additional cars are returned to the nationwide company, and thus disappear from the problem) and a
maximum of five cars can be moved from one location to the other in one night. We take the discount rate to be   = 0.9 and formulate this as a continuing finite MDP, where the time steps are days, the state is the number of cars at each location at the end of the day, and the actions are the net numbers of cars moved between the two locations overnight. Figure 4.2 shows the sequence of policies found by policy iteration starting from the policy that never moves any cars.

In [118]:
from scipy.stats import poisson
import numpy as np
import numba as nb
# poisson([1,3,4,5]).pmf(k=np.array([4, 5]).T)
# poisson.pmf(mu=np.array([3,4]),k=np.array([4, 5]), size=(2,2))

In [ ]:




def get_new_V(s, gamma = .9):
  """
  state update consists of
  - stochastic amount of new cars at each location (3 and 2)
  - stochastic amount of car rental requests (3 and 4)
  - deterministic change in amount of cars based on choice to move from 1 loc to another
  
  Reward signal:
  - deterministically based on action (- 2 dollar per moved car)
  - deterministic based on amount of rentals (10 per rented car)
  """

  # don't have to loop over actions, cause policy is deterministc
  # loop over all state transitions 


<ipython-input-29-fab789a404ec>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([(2,3), 2])


array([(2, 3), 2], dtype=object)

In [ ]:
r(s_prime, pi(s), s) 
allowable_a = clip(a)
morning_s = s + allowable_a
a_allowed

In [44]:
tuple(np.array([3,3]))

(3, 3)

In [45]:
np.array((2,3))

array([2, 3])

In [39]:
s = (2,3)
(s[0] + (-2, 3)

(2, 3, -2, 3)

In [ ]:
def get_r(s,a):
  reward_action = - 2 * abs(a)
  s_after_action = np.clip(np.array(s) + np.array([-a, a]) a_min=0, a_max=20)
  mor

In [142]:

def poisson_pmf(lam, k):
  return np.exp(-lam) * lam ** k  / np.math.factorial(k) 
poisson_pmf(3,2)

0.22404180765538775

In [143]:
def poisson_pmf(lam, k):
  return np.exp(-lam) * lam ** k  / np.math.factorial(k) 
poisson_pmf(3,2)

# @numba.jit(nopython=True)
# def trunc_poisson_jit(mu, k, k_max):
k_max = 3
[poisson_pmf(3, k=i) for i in range(k_max) ]

[0.049787068367863944, 0.14936120510359183, 0.22404180765538775]

In [131]:

def trunc_poisson(mu, k, k_max):  
    if k < k_max: 
      p = poisson.pmf(mu=3, k=k)
    elif (k == k_max):
      p = poisson.sf(mu=3, k = k-1)
    else:
      p = 0.0
    return p

def get_p_s_prime_1loc(cars_start, rental_req_lambda, car_returns_lambda):

  max_cars = 20
  p_s_prime_1loc = np.zeros(max_cars+1)

  for cars_rented_out in np.arange(cars_start+1):
    p_cars_rented_out = trunc_poisson(mu=rental_req_lambda, k=cars_rented_out, k_max=cars_start)

    cars_after_renting_out = cars_start - cars_rented_out
    max_new_cars = max_cars - cars_after_renting_out

    for cars_arrived in np.arange(max_new_cars+1):

      p_cars_arrived = trunc_poisson(mu=car_returns_lambda, k=cars_arrived, k_max=max_new_cars)
      cars_end = cars_after_renting_out + cars_arrived
      p_state_transition = p_cars_rented_out * p_cars_arrived
      p_s_prime_1loc[cars_end] += p_state_transition

  return p_s_prime_1loc

In [82]:
def get_prob_per_new_state(s, rental_request_lambdas, car_returns_lambdas):
  return np.outer(*[get_p_s_prime_1loc(s[i], rental_request_lambdas[i], car_returns_lambdas[i]) for i in [0, 1]])

get_prob_per_new_state((10,10), (3,4), (3,2)).sum()

1.0000000000000004

In [73]:
gamma = 0.9
max_cars=20
V = np.zeros((max_cars+1,max_cars+1))
(gamma*V*p_s_prime).sum()

0.0

In [83]:

def get_expected_R(s, rental_request_lambdas):
  expected_cars_rented_out = 0
  for cars_start, rental_req_lambda in zip(s, rental_request_lambdas):
    expected_cars_rented_out += np.array(
        [cars_rented_out * trunc_poisson(mu=rental_req_lambda, 
                                        k=cars_rented_out, 
                                        k_max=cars_start) 
        for cars_rented_out in np.arange(cars_start+1)]).sum()

    
  return 10 * expected_cars_rented_out

get_expected_R(s=(3,3), rental_request_lambdas=(4,3))

46.55749154067674

In [38]:
p_state_1loc_after_rental = 5 + trunc_poisson(mu, k, k_max):  


8

In [153]:
def get_s_after_action(s, action):
  return(tuple(np.clip(np.array(s) + np.array([-action, action]), a_min=0, a_max=20).astype('int'))
)

In [146]:
morn_to_eve_state_trans_matrices = {}
for s in np.ndindex(V.shape):
  morn_to_eve_state_trans_matrices[s] = get_prob_per_new_state(s, 
                                      rental_request_lambdas, 
                                      car_return_lambdas)

In [156]:
np.min([3, 2])

2

In [157]:
def get_action_space(s):

  max_to_second = np.min([5, s[0]])
  max_to_first = np.min([5, s[1]])
  return np.arange(-max_to_first, max_to_second + 1)

get_action_space((2,1))

array([-1,  0,  1,  2])

In [158]:
# %%time

# 1 init
V = np.random.uniform(size=(21,21))
A = np.arange(-5,6)
policy=np.zeros_like(V)
print(f"{V[:5,:5]=} \n\n {policy[:5,:5]=}")


transition_R = np.zeros_like(V)

rental_request_lambdas=(3,4)
car_return_lambdas = (3,2)
gamma = 0.9

# 2 
delta_theshold = 3

delta = delta_theshold + 1

P_new_state = get_prob_per_new_state(s_after_action, 
                                      rental_request_lambdas, 
                                      car_return_lambdas)

morn_to_eve_state_trans_matrices = {}
expected_rental_R = {}
for s in np.ndindex(V.shape):
  morn_to_eve_state_trans_matrices[s] = get_prob_per_new_state(s, 
                                      rental_request_lambdas, 
                                      car_return_lambdas)
  

  expected_rental_Rs[s] = get_expected_R(s, rental_request_lambdas)


# while delta >= delta_theshold:
for _ in range(12):
  delta = 0
  # for s in [(19,1)]:
  for s in np.random.permutation(list(np.ndindex(V.shape))):
    s = tuple(s)
    v = V[s]

    action = policy[s]
    s_after_action = get_s_after_action(s, action)

    expected_rental_R = expected_rental_Rs[s_after_action]
    P_new_state = morn_to_eve_state_trans_matrices[s_after_action]

    expected_V_new_state = (V*P_new_state).sum()

    action_R = - 2 * abs(action)
    V[s] =  expected_rental_R + action_R + gamma * expected_V_new_state

    delta = max(delta, abs(v-V[s]))

  print(f"sanity: {delta=}")


print(f"{V[:8,:8]=} \n\n {policy[:8,:8]=}")

# 3
# while delta >= delta_theshold:
for _ in range(20):
  changes = 0
  policy_stable = True
  for s in np.random.permutation(list(np.ndindex(V.shape))):
    s = tuple(s)
    old_action = policy[s]
    s_after_action = get_s_morning_start(s, action)
    old_actual_action = s_after_action[1] - s[1]
    old_action_R = - 2 * abs(old_actual_action)
    best_action = old_actual_action
    best_V = V[s]
    for alt_action in get_action_space(s):
      alt_s_in_morning = get_s_morning_start(s, alt_action)
      alt_actual_action = alt_s_in_morning[1] - s[1]
      alt_V = -2 * abs(alt_actual_action) + transition_R[alt_s_in_morning] + gamma*V[alt_s_in_morning]
      if alt_V > best_V:
        policy_stable = False
        best_V = alt_V
        policy[s] = alt_actual_action
        changes += 1

  print(f"sanity: {policy_stable}, {changes=}")

      


print(f"{V[:8,:8]=} \n\n {policy[:8,:8]=}")


V[:5,:5]=array([[0.19369787, 0.51843185, 0.30224921, 0.88218624, 0.64964787],
       [0.92491568, 0.10550145, 0.43364549, 0.23963957, 0.00099242],
       [0.27602   , 0.37083503, 0.06061199, 0.45367463, 0.59972001],
       [0.37889626, 0.70777902, 0.5310784 , 0.86370163, 0.04644563],
       [0.73509355, 0.58612066, 0.61826414, 0.07781606, 0.93967588]]) 

 policy[:5,:5]=array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])


NameError: ignored

In [108]:
a = np.ndindex(V.shape)
a